In [1]:
import pandas as pd

In [2]:
# sqlite3_load_csv.py

import csv
import sqlite3
import sys
import glob

In [3]:
%load_ext sql
%load_ext lab_black

# Get data

In [4]:
from pathlib import Path
import sys

## import custom functions

# Create path to custom functions
funcs_path = Path.cwd().parents[1]
sys.path.insert(1, str(funcs_path))

from funcs import paths

In [5]:
## Get path to raw data

path_to_SM_electricity_raw_data_folder = paths.get_path_to_folder("SM_electricity")

path_to_SM_electricity_raw_data_folder

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity'

In [6]:
## Get path to processed data folder

path_to_processed_data_folder = paths.get_path_to_folder("processed")

path_to_processed_data_folder

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\processed'

In [7]:
path_to_data = path_to_SM_electricity_raw_data_folder + r"\File"
all_files = glob.glob(path_to_data + "*.txt")

display(path_to_data, all_files)

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File'

['C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File1.txt',
 'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File2.txt',
 'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File3.txt',
 'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File4.txt',
 'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File5.txt',
 'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\raw\\SM_electricity\\File6.txt']

In [8]:
path_to_output_data = path_to_processed_data_folder + r"\data.db"

path_to_output_data

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\processed\\data.db'

# Bulk load txt files

Code skeleton
https://pymotw.com/3/sqlite3/

Executemany docs (insert multiple rows at once)
https://kite.com/python/docs/sqlite3.Cursor.executemany

In [9]:
%sql sqlite:///$path_to_output_data

'Connected: @C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\processed\\data.db'

In [10]:
db_filename = path_to_output_data

conn = sqlite3.connect(db_filename)  # or use :memory: to put it in RAM
cursor = conn.cursor()

# create a table
cursor.execute(
    """CREATE TABLE IF NOT EXISTS electricity_data_raw (
            ID TEXT,
            Date_ID INTEGER,
            Consumption FLOAT
        ); 
               """
)

In [12]:
%%time

SQL_insert_query = """
INSERT INTO electricity_data_raw 
(ID, Date_ID, Consumption)
VALUES (?,?,?);
"""

for input_file in all_files:

    """ Can only look at file data within with block as outside it the file is closed"""
    with open(input_file, "rt") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=" ")

        with sqlite3.connect(db_filename) as conn:
            cursor = conn.cursor()
            cursor.executemany(SQL_insert_query, csv_reader)

Wall time: 8min 20s


In [14]:
%%sql 
SELECT * FROM electricity_data_raw
LIMIT 5

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.


ID,Date_ID,Consumption
1392,19503,0.14
1392,19504,0.138
1392,19505,0.14
1392,19506,0.145
1392,19507,0.145
